# 1. Split the document.
# 2. Separate the document.
#     - Cannot generate the answer due to exceeding the number of tokens.
#     - Generation time depends on the length of the document.
# 3. Embed and save to the vector database.
# 4. Query: perform similarity search on the vector database.
# 5. Pass the documents from the similarity search to the LLM.

In [11]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    
    chunk_overlap=200, 
)

loader = Docx2txtLoader("./tax_with_markdown.docx")
document_lists = loader.load_and_split(text_splitter=text_splitter)
len(document_lists)

225

In [12]:
document_lists[52]

Document(metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한

In [13]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [14]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv

load_dotenv()

index_name = 'tax-markdown-index'
pinecone_api_key = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(
    documents=document_lists,
    embedding=embedding,
    index_name=index_name
)

In [72]:
query = '연봉이 5000만원인 직장인은 소득세로 얼마를 내야하나요?'

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [17]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\wjdtp\miniconda3\envs\langchain-basic\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [23]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

[Document(id='96804e0f-4ed8-4f44-9eaf-90d8b79deb5e', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [24]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)
ai_msg = qa_chain.invoke({"query": query})

In [25]:
ai_msg

{'query': '연봉이 5000만원인 직장인은 소득세로 얼마를 내야하나요?',
 'result': '연봉이 5000만원인 직장인의 소득세는 기본세율에 따라 계산됩니다. 5,000만원 이하 소득인 경우, 1,400만원 초과 5,000만원 이하의 과세표준에 대해 84만원 + (1,400만원을 초과하는 금액의 15%) 이므로, 소득세는 약 582만원입니다.'}

In [ ]:
query = '연봉이 5000만원인 거주자의 종합소득세로 얼마를 내야하나요?'

In [27]:
retriever.invoke(query)

[Document(id='e503bf14-b69f-4cf2-9b7e-2271f3495f4e', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소

In [28]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
)
ai_msg = qa_chain.invoke({"query": query})

In [48]:
ai_msg

{'query': '연봉이 5000만원인 거주자는 소득세로 얼마를 내야하나요?',
 'result': '연봉이 5,000만원인 거주자의 소득세는 1,400만원을 초과하는 금액에 대해 15%가 적용됩니다. 계산식은 84만원 + (5,000만원 - 1,400만원) * 15%입니다. 따라서 소득세는 총 606만원이 됩니다.'}

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
     사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경 할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}                
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [73]:
query

'연봉이 5000만원인 직장인은 소득세로 얼마를 내야하나요?'

In [78]:
new_question = dictionary_chain.invoke({"question": query})

In [79]:
new_question

'연봉이 5000만원인 거주자는 소득세로 얼마를 내야하나요?'

In [57]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [58]:
ai_response = tax_chain.invoke({"question": query})

In [59]:
ai_response

{'query': '연봉이 5000만원인 거주자의 종합소득세로 얼마를 내야하나요?',
 'result': '연봉이 5,000만 원인 거주자의 종합소득세는 624만 원입니다. 이는 세율에 따라 5,000만 원 초과 부분에 대한 계산 없이 기본 금액만 적용된 결과입니다.'}